In [5]:
import gym
env = gym.envs.make("MountainCarContinuous-v0") 

[2020-04-17 12:57:30,425] Making new env: MountainCarContinuous-v0
/home/soham/.local/lib/python3.6/site-packages/gym/envs/registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [6]:
env.observation_space.sample()

array([-0.21213569,  0.03012651])

In [7]:
initial_state = env.reset()
initial_state

array([-0.59724516,  0.        ])

In [8]:
import numpy as np
test_action = np.array([0.5])
env.step(test_action)

(array([-0.5959473 ,  0.00129787]), -0.025, False, {})

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
import torch.optim as optim
import random
import copy
from collections import namedtuple, deque
import matplotlib.pyplot as plt

In [10]:
norm = Normal(torch.tensor([4.0]), torch.tensor([0.5]))

In [11]:
norm.sample()

tensor([5.2487])

In [15]:
import tensorflow as tf
norm1 = tf.contrib.distributions.Normal(4.0, 0.5)

In [18]:
norm1.sample(1)

<tf.Tensor 'Normal_2/sample/Reshape:0' shape=(1,) dtype=float32>

In [12]:
class Critic(nn.Module):
    def __init__(self):
        super(Critic, self).__init__()
        n_hidden1 = 400
        n_hidden2 = 400
        n_out = 1
        self.size = 2
        self.fc1 = nn.Linear(self.size, n_hidden1)
        self.fc2 = nn.Linear(n_hidden1, n_hidden2)
        self.fc3 = nn.Linear(n_hidden2,n_out)
        #self.sig = nn.Sigmoid()
        
    def forward(self, x):
        x = F.relu(self.fc1(x))  
        x = F.relu(self.fc2(x))  
        x = self.fc3(x)
        return x
    
class Actor(nn.Module):
    def __init__(self):
        super(Actor,self).__init__()
        n_hidden1 = 40;
        n_hidden2 = 40
        n_out = 1
        self.size = 2
        self.fc1 = nn.Linear(self.size,n_hidden1)
        self.fc2 = nn.Linear(n_hidden1, n_hidden2)
        self.fc3 = nn.Linear(n_hidden2,n_out)
        self.fc4 = nn.Linear(n_hidden2,n_out)
    
    def forward(self,state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        mu = self.fc3(x)
        sigma = self.fc3(x)
        norm_dist = Normal(mu,sigma)
        action = norm_dist.sample()
        action = torch.clamp(action, env.action_space.low[0], env.action_space.high[0], out=None)
        return action,norm_dist

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cuda', index=0)

In [13]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 128        # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR_ACTOR = 1e-4         # learning rate of the actor 
LR_CRITIC = 1e-3        # learning rate of the critic
WEIGHT_DECAY = 0        # L2 weight decay

In [60]:
import sklearn
import sklearn.preprocessing

state_space_samples = np.array(
    [env.observation_space.sample() for x in range(10000)])
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(state_space_samples)

#function to normalize states
def scale_state(state):                  #requires input shape=(2,)
    scaled = scaler.transform([state])
    return scaled

In [14]:
class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
        self.state = x + dx
        return self.state

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

In [37]:
def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()
def weight_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        xavier(m.weight.data)
        xavier(m.bias.data)

In [57]:
class Agent():
    def __init__(self):
        self.state_size = 2
        self.action_size = 1
        #self.seed = random.seed(random_seed)
        self.actor_local = Actor().to(device)
        self.actor_target = Actor().to(device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=LR_ACTOR)
        
        self.critic_local = Critic().to(device)
        self.critic_target = Critic().to(device)
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr=LR_CRITIC, weight_decay=WEIGHT_DECAY)
        
        # Noise process
        self.noise = OUNoise(self.action_size, 2)

        # Replay memory
        self.memory = ReplayBuffer(self.action_size, BUFFER_SIZE, BATCH_SIZE, 2)
    def step(self, state, action, reward, next_state, done):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        # Save experience / reward
        self.memory.add(scale_state(state), action, reward, scale_state(next_state), done)

        # Learn, if enough samples are available in memory
        if len(self.memory) > BATCH_SIZE:
            experiences = self.memory.sample()
            self.learn(experiences, GAMMA)
    def reset(self):
        self.noise.reset()
        self.actor_local.apply(weight_reset)
        self.actor_target.apply(weight_reset)
        self.critic_local.apply(weight_reset)
        self.critic_target.apply(weight_reset)
        #self.actor_optimizer.apply(weight_reset)
        #self.critic_optimizer.apply(weight_reset)
        self.actor_local.apply(weight_init)
        self.actor_target.apply(weight_init)
        self.critic_local.apply(weight_init)
        self.critic_target.apply(weight_init)
        #self.actor_optimizer.apply(weight_init)
        #self.critic_optimizer.apply(weight_init)
    def act(self, state, add_noise=True):
        """Returns actions for given state as per current policy."""
        state = torch.from_numpy(state).float().to(device)
        self.actor_local.eval()
        with torch.no_grad():
            action = self.actor_local(state)[0].cpu().data.numpy()
        self.actor_local.train()
        if add_noise:
            action += self.noise.sample()
        return np.clip(action, -1, 1)
    def learn(self, experiences, gamma):
        """Update policy and value parameters using given batch of experience tuples.
        Q_targets = r + γ * critic_target(next_state, actor_target(next_state))
        where:
            actor_target(state) -> action
            critic_target(state, action) -> Q-value

        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # ---------------------------- update critic ---------------------------- #
        # Get predicted next-state actions and Q values from target models
        Q_targets_next = self.critic_target(next_states)
        # Compute Q targets for current states (y_i)
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        # Compute critic los
        Q_expected = self.critic_local(states)
        td_error = Q_targets - Q_expected
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.critic_optimizer.zero_grad()
        critic_loss.backward(retain_graph=True)
        self.critic_optimizer.step()

        # ---------------------------- update actor ---------------------------- #
        # Compute actor loss
        actions_pred,norm_dist = self.actor_local(states)
        actor_loss = -(norm_dist.log_prob(actions_pred)*td_error).mean()
        #print(actor_loss)
        # Minimize the loss
        self.actor_optimizer.zero_grad()
        actor_loss.backward(retain_graph=True)
        self.actor_optimizer.step()

        # ----------------------- update target networks ----------------------- #
        self.soft_update(self.critic_local, self.critic_target, TAU)
        self.soft_update(self.actor_local, self.actor_target, TAU)                     

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model: PyTorch model (weights will be copied from)
            target_model: PyTorch model (weights will be copied to)
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

In [58]:
agent = Agent()

In [ ]:
flag = 0
def actor_critic(n_episodes=1000, max_t=300, print_every=100):
    flag = 0
    scores_deque = deque(maxlen=print_every)
    scores = []
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        agent.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            if reward >=100:
                flag = 1
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_deque.append(score)
        scores.append(score)
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        if i_episode % print_every == 0:
            if flag == 0:
                return scores,flag
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            
    return scores,flag

scores,flag = actor_critic(print_every = 25)
while flag == 0:
    print("restarting")
    agent.reset()
    scores,flag = actor_critic(print_every = 25)

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 25	Average Score: -13.35restarting
Episode 25	Average Score: -13.06restarting
Episode 25	Average Score: -12.52restarting
Episode 25	Average Score: -13.80restarting
Episode 25	Average Score: -13.76restarting
Episode 25	Average Score: -15.58restarting
Episode 25	Average Score: -13.67restarting
Episode 25	Average Score: -13.49restarting
Episode 25	Average Score: -15.03restarting
Episode 25	Average Score: -13.12restarting
Episode 25	Average Score: -13.42restarting
Episode 25	Average Score: -14.49restarting
Episode 25	Average Score: -15.50restarting
Episode 25	Average Score: -13.21restarting
Episode 25	Average Score: -14.00restarting
Episode 25	Average Score: -13.28restarting
Episode 25	Average Score: -13.77restarting
Episode 25	Average Score: -13.01restarting
Episode 25	Average Score: -13.46restarting
Episode 25	Average Score: -14.55restarting
Episode 25	Average Score: -14.71restarting
Episode 25	Average Score: -15.39restarting
Episode 25	Average Score: -14.24restarting
Episode 25	